### Extract the geoboundry of the Indian cities

In [12]:
import pandas as pd
import osmnx as ox
import geopandas as gpd
from tqdm import tqdm

In [13]:
df = pd.read_csv("AirQuality/Dataset/Ground_Truth_2023_Final.csv")

cities_df = df[['city', 'state']].drop_duplicates().reset_index(drop=True)

cities_df['query'] = cities_df['city'] + ", " + cities_df['state'] + ", India"

In [14]:
tqdm.pandas()

def get_city_boundry(query):
    try:
        gdf = ox.geocode_to_gdf(query)
        return gdf.loc[0,'geometry']
    except:
        return None
    
cities_df['geometry'] = cities_df['query'].progress_apply(get_city_boundry)

100%|██████████| 201/201 [08:16<00:00,  2.47s/it]


In [15]:
gdf = gpd.GeoDataFrame(cities_df, geometry='geometry')
gdf = gdf.set_crs("EPSG:4326")  # WGS84

In [16]:
gdf.to_file("city_boundaries.geojson", driver="GeoJSON")

In [17]:
gdf1 = gpd.read_file("AirQuality/RQ3/Dataset/city_boundaries.geojson")
gdf2 = gpd.read_file("AirQuality/RQ3/Dataset/indian_cities_with_fallback.geojson")

gdf1 = gdf1.to_crs("EPSG:4326")
gdf2 = gdf2.to_crs("EPSG:4326")

In [18]:
print(gdf1.columns)
print(gdf1.head())

Index(['city', 'state', 'query', 'geometry'], dtype='object')
        city          state                       query  \
0   Agartala        Tripura    Agartala, Tripura, India   
1       Agra  Uttar Pradesh  Agra, Uttar Pradesh, India   
2  Ahmedabad        Gujarat   Ahmedabad, Gujarat, India   
3     Aizawl        Mizoram      Aizawl, Mizoram, India   
4      Ajmer      Rajasthan     Ajmer, Rajasthan, India   

                                            geometry  
0                                               None  
1  POLYGON ((77.85102 27.15764, 77.85103 27.15734...  
2  POLYGON ((71.83998 22.32574, 71.84003 22.32543...  
3  POLYGON ((92.62402 23.7976, 92.62543 23.79614,...  
4  POLYGON ((74.5916 26.48066, 74.59576 26.46199,...  


In [19]:
print(gdf2.columns)
print(gdf2.head())

Index(['city', 'state', 'source', 'bbox_west', 'bbox_south', 'bbox_east',
       'bbox_north', 'place_id', 'osm_type', 'osm_id', 'lat', 'lon', 'class',
       'type', 'place_rank', 'importance', 'addresstype', 'name',
       'display_name', 'geometry'],
      dtype='object')
          city         state   source  bbox_west  bbox_south  bbox_east  \
0       Bhilai  Chhattisgarh   buffer        NaN         NaN        NaN   
1       Raipur  Chhattisgarh  polygon  81.533994   20.940078  82.194729   
2       Jaipur     Rajasthan  polygon  75.481482   26.672196  75.938573   
3  Navi Mumbai   Maharashtra  polygon  72.977859   18.998448  73.049704   
4         Pune   Maharashtra  polygon  73.749847   18.429497  74.020214   

   bbox_north     place_id  osm_type      osm_id        lat        lon  \
0         NaN          NaN      None         NaN        NaN        NaN   
1   21.625935  231098222.0  relation   1972682.0  21.280909  81.816407   
2   27.057694  226980493.0  relation   1950062.0  2

In [20]:
# Step 1: Identify cities missing geometry in gdf1
missing_cities = gdf1[gdf1['geometry'].isna()]['city'].tolist()

# Step 2: Filter gdf2 to get boundaries for only those missing cities
fallback_boundaries = gdf2[gdf2['city'].isin(missing_cities)].copy()

# Now fallback_boundaries contains boundaries from gdf2 only for missing cities in gdf1

# Optional: Check how many missing cities got boundaries from gdf2
print(f"Number of missing cities in gdf1: {len(missing_cities)}")
print(f"Number of fallback boundaries found in gdf2: {len(fallback_boundaries)}")

# fallback_boundaries can be used separately or saved as needed

Number of missing cities in gdf1: 35
Number of fallback boundaries found in gdf2: 32


In [21]:
geom_dict = gdf2.set_index('city')['geometry'].to_dict()

def fill_geometry(row):
    if row['geometry'] is None or row['geometry'].is_empty:
        # Try to get geometry from gdf2
        fallback_geom = geom_dict.get(row['city'], None)
        return fallback_geom
    else:
        return row['geometry']

gdf1['geometry'] = gdf1.apply(fill_geometry, axis=1)

missing_after_fill = gdf1['geometry'].isna().sum()
print(f"Number of missing geometries after fill: {missing_after_fill}")

Number of missing geometries after fill: 4


In [22]:
gdf1.to_file("city_boundaries_all.geojson", driver="GeoJSON")

### Collect NDVI values

In [1]:
import requests
from bs4 import BeautifulSoup
import os
import xarray as xr
import geopandas as gpd
import rasterio.features
import numpy as np
import pandas as pd
from shapely.geometry import mapping
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

In [ ]:
base_url = "https://www.ncei.noaa.gov/data/land-normalized-difference-vegetation-index/access/2023/"

download_dir = "ndvi_2023"
os.makedirs(download_dir,exist_ok=True)

response = requests.get(base_url)
soup = BeautifulSoup(response.text,'html.parser')

nc_files = [a['href'] for a in soup.find_all('a',href=True) if a['href'].endswith('.nc')]

for nc_file in nc_files:
    file_url = base_url + nc_file
    local_path = os.path.join(download_dir, nc_file)
    if not os.path.exists(local_path):
        print(f'Downloading {nc_file}...')
        with requests.get(file_url, stream=True) as r:
            r.raise_for_status()
            with open(local_path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
    else:
        print(f'{nc_file} already exists. Skipping download.')


In [24]:
city_geojson_path = "AirQuality/RQ3/Dataset/city_boundaries_all.geojson"
ndvi_files_dir = "AirQuality/RQ3/Dataset/ndvi_2023"

cities_gdf = gpd.read_file(city_geojson_path)

sample_file = sorted([f for f in os.listdir(ndvi_files_dir) if f.endswith(".nc")])[0]
sample_ds = xr.open_dataset(os.path.join(ndvi_files_dir, sample_file))
sample_ndvi = sample_ds["NDVI"]

lats = sample_ndvi.latitude.values
lons = sample_ndvi.longitude.values

pixel_size_x = lons[1] - lons[0]
pixel_size_y = lats[0] - lats[1]

transform = rasterio.transform.from_origin(
    west=lons.min() - pixel_size_x / 2,
    north=lats.max() + pixel_size_y / 2,
    xsize=pixel_size_x,
    ysize=pixel_size_y,
)

height = len(lats)
width = len(lons)

sample_ds.close()

In [26]:
def create_mask(geom):
    if geom is None:
        return None
    return rasterio.features.rasterize(
        [(mapping(geom), 1)],
        out_shape=(height, width),
        transform=transform,
        fill=0,
        all_touched=True,
        dtype=np.uint8,
    )

print("Creating masks for cities...")
cities_gdf['mask'] = cities_gdf['geometry'].apply(create_mask)


Creating masks for cities...


In [ ]:
def process_file(filename):
    filepath = os.path.join(ndvi_files_dir, filename)

    try:
        ds = xr.open_dataset(filepath, mask_and_scale=False)
        ndvi_data = ds["NDVI"].values[0]  # (lat, lon)
        ds.close()

        date_str = filename.split("_")[4]
        date = pd.to_datetime(date_str, format="%Y%m%d")

        results = []
        for idx, row in cities_gdf.iterrows():
            masked_values = np.where(row['mask'] == 1, ndvi_data, np.nan)
            mean_ndvi = np.nanmean(masked_values)
            results.append({
                "city": row["city"],
                "state": row["state"],
                "date": date,
                "mean_ndvi": mean_ndvi
            })
        print(f"Processed file: {filename} — {len(results)} cities")
        return results

    except Exception as e:
        print(f"Failed to process file: {filename} — {e}")
        failed_files.append(filename)
        return []  

files = sorted([f for f in os.listdir(ndvi_files_dir) if f.endswith(".nc")])
print(f"Processing {len(files)} files with {cpu_count()} cores...")

failed_files = []

with Pool() as pool:
    all_results = list(tqdm(pool.imap(process_file, files), total=len(files)))

# Flatten results and convert to DataFrame
flat_results = [item for sublist in all_results for item in sublist]
final_df = pd.DataFrame(flat_results)

# final_df.to_csv("city_daily_ndvi_2023.csv", index=False)
# print("Done! Saved to 'city_daily_ndvi_2023.csv'")

>Scale down NDVI values using scale factor 

In [30]:
filepath = "AirQuality/RQ3/Dataset/ndvi_2023/VIIRS-Land_v001_JP113C1_NOAA-20_20230101_c20240123205954.nc"

ds = xr.open_dataset(filepath,mask_and_scale=False)

print(ds["NDVI"])
print(ds["NDVI"].attrs)

for at in ds.attrs:
    print(f"{at} = {ds.attrs[at]}")

<xarray.DataArray 'NDVI' (time: 1, latitude: 3600, longitude: 7200)> Size: 52MB
[25920000 values with dtype=int16]
Coordinates:
  * latitude   (latitude) float32 14kB 89.97 89.93 89.88 ... -89.93 -89.97
  * longitude  (longitude) float32 29kB -180.0 -179.9 -179.9 ... 179.9 180.0
  * time       (time) datetime64[ns] 8B 2023-01-01
Attributes:
    scale_factor:   0.0001
    add_offset:     0.0
    long_name:      NOAA Climate Data Record of Normalized Difference Vegetat...
    units:          1
    valid_range:    [-1000 10000]
    _FillValue:     -9999
    grid_mapping:   crs
    standard_name:  normalized_difference_vegetation_index
{'scale_factor': np.float64(0.0001), 'add_offset': np.float64(0.0), 'long_name': 'NOAA Climate Data Record of Normalized Difference Vegetation Index', 'units': '1', 'valid_range': array([-1000, 10000], dtype=int16), '_FillValue': np.int16(-9999), 'grid_mapping': 'crs', 'standard_name': 'normalized_difference_vegetation_index'}
title = Normalized Difference V

In [32]:
ndvi_df = final_df
print(ndvi_df.head())

        city          state       date    mean_ndvi
0   Agartala        Tripura 2023-01-01  5176.909091
1       Agra  Uttar Pradesh 2023-01-01  5895.162162
2  Ahmedabad        Gujarat 2023-01-01  4059.651515
3     Aizawl        Mizoram 2023-01-01  8331.448276
4      Ajmer      Rajasthan 2023-01-01  3804.250000


In [33]:
ndvi_raw = ds["NDVI"].values.astype(float)
scale_factor = ds["NDVI"].attrs.get("scale_factor",1.0)
fill_value = ds["NDVI"].attrs.get("_FillValue",None)

In [34]:
ndvi_df['mean_ndvi'] = ndvi_df['mean_ndvi'].apply(lambda x:np.nan if x == -9999 else x*scale_factor)

In [35]:
ndvi_df

,city,state,date,mean_ndvi
0,Agartala,Tripura,2023-01-01,0.517691
1,Agra,Uttar Pradesh,2023-01-01,0.589516
2,Ahmedabad,Gujarat,2023-01-01,0.405965
3,Aizawl,Mizoram,2023-01-01,0.833145
4,Ajmer,Rajasthan,2023-01-01,0.380425
...,...,...,...,...
72958,Vijayawada,Andhra Pradesh,2023-12-31,0.283200
72959,Visakhapatnam,Andhra Pradesh,2023-12-31,0.058298
72960,Vrindavan,Uttar Pradesh,2023-12-31,0.198505
72961,Yadgir,Karnataka,2023-12-31,0.404346


In [36]:
ndvi_df['date'] = pd.to_datetime(ndvi_df['date'])
ndvi_df['month'] = ndvi_df['date'].dt.to_period("M")

In [37]:
monthly_ndvi_df = ndvi_df.groupby(['city', 'state', 'month']).agg(
    avg_ndvi=('mean_ndvi', 'mean'),
    max_ndvi=('mean_ndvi', 'max'),
    min_ndvi=('mean_ndvi', 'min'),
    ndvi_range=('mean_ndvi', lambda x: x.max() - x.min())
).reset_index()

In [38]:
monthly_ndvi_df

,city,state,month,avg_ndvi,max_ndvi,min_ndvi,ndvi_range
0,Agartala,Tripura,2023-01,0.363576,0.517691,0.008791,0.508900
1,Agartala,Tripura,2023-02,0.391827,0.555423,-0.005073,0.560495
2,Agartala,Tripura,2023-03,0.435666,0.710977,0.006709,0.704268
3,Agartala,Tripura,2023-04,0.513448,0.642455,-0.000300,0.642755
4,Agartala,Tripura,2023-05,0.322064,0.614464,-0.030768,0.645232
...,...,...,...,...,...,...,...
2407,Yamuna Nagar,Haryana,2023-08,0.321059,0.701923,-0.014614,0.716536
2408,Yamuna Nagar,Haryana,2023-09,0.452018,0.707345,-0.014368,0.721714
2409,Yamuna Nagar,Haryana,2023-10,0.414028,0.562855,-0.013209,0.576064
2410,Yamuna Nagar,Haryana,2023-11,0.293724,0.429895,-0.016486,0.446382


In [15]:
yearly_ndvi_df = ndvi_df.groupby(['city', 'state']).agg(
    avg_ndvi=('mean_ndvi', 'mean'),
    max_ndvi=('mean_ndvi', 'max'),
    min_ndvi=('mean_ndvi', 'min'),
    ndvi_range=('mean_ndvi', lambda x: x.max() - x.min())
).reset_index()

In [17]:
yearly_ndvi_df

,city,state,avg_ndvi,max_ndvi,min_ndvi,ndvi_range
0,Agra,Uttar Pradesh,0.252453,0.640851,-0.232786,0.873638
1,Ahmedabad,Gujarat,0.256610,0.503166,-0.102508,0.605674
2,Aizawl,Mizoram,0.471525,0.895637,-0.053691,0.949328
3,Ajmer,Rajasthan,0.297504,0.530650,-0.756600,1.287250
4,Alwar,Rajasthan,0.308324,0.625097,-0.330223,0.955320
...,...,...,...,...,...,...
139,Vijayapura,Karnataka,0.237465,0.534435,-0.035784,0.570219
140,Visakhapatnam,Andhra Pradesh,0.194100,0.536722,-0.184405,0.721128
141,Vrindavan,Uttar Pradesh,0.297198,0.679038,-0.095129,0.774167
142,Yadgir,Karnataka,0.287445,0.658752,-0.030703,0.689455


In [39]:
monthly_ndvi_df.to_csv("city_monthly_ndvi_2023.csv",index=False)

In [18]:
yearly_ndvi_df.to_csv("city_yearly_ndvi_2023.csv",index=False)